<a href="https://colab.research.google.com/github/crazyhamster09/e/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled2_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

import os
os.environ['KAGGLE_USERNAME'] = "juliaya" # username from the json file
os.environ['KAGGLE_KEY'] = "f4a7d9ccd189b35078dc6202fa26f882"
!kaggle competitions download -c news-scraping-competition


 99% 22.0M/22.1M [00:02<00:00, 8.88MB/s]
100% 22.1M/22.1M [00:02<00:00, 10.8MB/s]


In [ ]:
!unzip /content/news-scraping-competition.zip

Archive:  /content/news-scraping-competition.zip
  inflating: base_submission_news.csv  
  inflating: test_news.csv           


In [1]:
!pip install bs4
!pip install openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=f09f1f8f476acc2d49b0d40b103b14cc2db3b5b40b329da56edf9721d4673b70
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [3]:
class lentaRu_parser:
    def __init__(self):
        pass


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
​
        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней
​
        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК
​
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [4]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "1"
dateFrom = "2010-01-01"

dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2010-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '1', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 100,
                         save_every = 1,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Parsing articles from 2012-06-28 to 2012-10-06


<ipython-input-3-2bfa07dc550d>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


In [ ]:
df = tbl[['text', 'bloc']]
df.head

NameError: name 'tbl' is not defined

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "3"
dateFrom = "2010-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2010-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '3', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 100,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Parsing articles from 2023-10-30 to 2024-02-01


<ipython-input-6-2bfa07dc550d>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Finish
48888


,rightcol,docid,lastmodtime,part,title,type,url,tags,bloc,domain,modified,image_url,text,status,pubdate,snippet
0,В Литве выключили атомную электростанцию,158,1670590771,0,Игналинская АЭС прекратила работу,1,https://lenta.ru/news/2010/01/01/aes/,[],3,1,1262310838,https://icdn.lenta.ru/images/0000/0282/0000028...,Игналинская АЭС. Фото с сайта coe.int Игналинс...,0,1262310838,Игналинская АЭС. Фото с сайта coe.int Игналинс...
1,Председателем ОБСЕ стал Канат Саудабаев,183,1670590773,0,Председательство в ОБСЕ перешло к Казахстану,1,https://lenta.ru/news/2010/01/01/osce/,[],3,1,1262328917,https://icdn.lenta.ru/images/0000/0282/0000028...,Канат Саудабаев (слева) и Георгиос Папандреу. ...,0,1262328917,Канат Саудабаев (слева) и Георгиос Папандреу. ...
2,Студенты вуза смогут продолжить высшее професс...,243,1670590777,0,В Латвии закрыли полицейскую академию,1,https://lenta.ru/news/2010/01/01/shutdown/,[],3,1,1262376609,https://icdn.lenta.ru/images/0000/0282/0000028...,"Фото Никиты Кузьмина, Телеграф.lv Академия пол...",0,1262376609,"Фото Никиты Кузьмина, Телеграф.lv ... свою дея..."
3,Литва нашла средства для участия в песенном ко...,264,1670590778,0,"Литва нашла спонсора для участия в ""Евровидени...",1,https://lenta.ru/news/2010/01/01/find/,[],3,1,1262388068,https://icdn.lenta.ru/images/0000/0282/0000028...,"Логотип ""Евровидения-2010"". Изображение с сайт...",0,1262388068,"Логотип ""Евровидения-2010"". Изображение с сайт..."
4,Данных о пострадавших и разрушениях в результа...,294,1670590781,0,В Таджикистане произошло землетрясение магниту...,1,https://lenta.ru/news/2010/01/02/quake/,[],3,1,1262419446,https://icdn.lenta.ru/images/0000/0282/0000028...,Спутниковый снимок территории Таджикистана. Фо...,0,1262419446,Спутниковый снимок территории Таджикистана. Фо...


In [ ]:
df2 = tbl[['text', 'bloc']]
df2.head

<bound method NDFrame.head of                                                     text  bloc
0      Игналинская АЭС. Фото с сайта coe.int Игналинс...     3
1      Канат Саудабаев (слева) и Георгиос Папандреу. ...     3
2      Фото Никиты Кузьмина, Телеграф.lv Академия пол...     3
3      Логотип "Евровидения-2010". Изображение с сайт...     3
4      Спутниковый снимок территории Таджикистана. Фо...     3
...                                                  ...   ...
48883  Фото: Nacho Doce / Reuters Варвара Кошечкина В...     3
48884  Фото: Stringer / Reuters Олег Давыдов Украинск...     3
48885  Фото: Gleb Garanich / Reuters Игорь Дмитров В ...     3
48886  Фото: РИА Новости Игорь Дмитров Армения предла...     3
48887  Фото: Federation Council of Russia / Globalloo...     3

[48888 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "4"
dateFrom = "2010-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2010-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '4', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Parsing articles from 2024-01-18 to 2024-02-01


<ipython-input-6-2bfa07dc550d>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Finish
85197


,rightcol,docid,lastmodtime,part,title,type,url,tags,bloc,domain,modified,image_url,text,status,pubdate,snippet
0,C 1 января 2010 года пол-литровая бутылка не м...,214,1670590775,0,Приказ о минимальной цене на водку вступил в силу,1,https://lenta.ru/news/2010/01/01/vodka/,[],4,1,1262357129,https://icdn.lenta.ru/images/0000/0282/0000028...,"Кадр ТВ-6, архив C 1 января 2010 года в России...",0,1262357129,"Кадр ТВ-6, архив C 1 января 2010 года в ... пр..."
1,Подписчики Time Warner Cable не остались без т...,297,1670590781,0,Fox и Time Warner Cable согласовали плату за т...,1,https://lenta.ru/news/2010/01/02/deal/,[],4,1,1262423698,https://icdn.lenta.ru/images/0000/0282/0000028...,Штаб-квартира Time Warner в Нью-Йорке. Фото (c...,0,1262423698,Штаб-квартира Time Warner в Нью-Йорке. Фото (c...
2,Четверть граждан Исландии подписали петицию пр...,383,1670590788,0,Исландцы выразили протест против выплат компен...,1,https://lenta.ru/news/2010/01/03/iceland/,[],4,1,1262488184,https://icdn.lenta.ru/images/0000/0282/0000028...,Штаб-квартира Landsbanki. Фото (c)AFP Четверть...,0,1262488184,Штаб-квартира Landsbanki. Фото (c)AFP Четверть...
3,На транзите топлива это пока не сказалось,492,1670590797,0,Трейдеры сообщили о прекращении поставок росси...,1,https://lenta.ru/news/2010/01/03/oil/,[],4,1,1262546946,https://icdn.lenta.ru/images/0000/0282/0000028...,Мозырский НПЗ. Фото (c)AFP Россия прекратила п...,0,1262546946,Мозырский НПЗ. Фото (c)AFP Россия прекратила п...
4,Переговоры продолжатся в понедельник,511,1670590798,0,России и Белоруссии пока не удалось договорить...,1,https://lenta.ru/news/2010/01/03/goon/,[],4,1,1262558717,https://icdn.lenta.ru/images/0000/0282/0000028...,Датчик давления на нефтепроводе в Белоруссии. ...,0,1262558717,Датчик давления на нефтепроводе в Белоруссии. ...


In [ ]:
df3 = tbl[['text', 'bloc']]
df3.head

<bound method NDFrame.head of                                                     text  bloc
0      Фото: Rupert Oberhäuser / Globallookpress.com ...     4
1      Фото: Константин Кокошкин / «Коммерсантъ» Росс...     4
2      Фото: Depositphotos Покупать в интернете за гр...     4
3      Фото: Sean Gallup / Getty Images Цена на газ д...     4
4      Фото: Олег Харсеев / «Коммерсантъ» В России ст...     4
...                                                  ...   ...
36918  Фото: Константин Михальчевский / РИА Новости М...     4
36919  Фото: Komsomolskaya Pravda / AP Варвара Кошечк...     4
36920  Фото: Unsplash Семен Александров Финляндия про...     4
36921  Фото: Raheb Homavandi / Reuters Варвара Кошечк...     4
36922  Фото: Министерство обороны / РИА Новости Марин...     4

[36923 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "37"
dateFrom = "2010-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '37', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
29647


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943858,https://lenta.ru/news/2020/01/01/magas/,Стало известно о состоянии пострадавших при на...,1577874120,1577874120,1,1,0,0,37,[136],https://icdn.lenta.ru/images/2020/01/01/13/202...,1577874120,Фото: Наталья Селиверстова / РИА Новости Состо...,Опасности для жизни раненых полицейских нет,"Фото: Наталья Селиверстова / РИА Новости ...),..."
1,943863,https://lenta.ru/news/2020/01/01/sarapul/,Россиянин ударил ножом прохожих за просьбу дат...,1577875867,1577875867,1,1,0,0,37,[135],https://icdn.lenta.ru/images/2020/01/01/13/202...,1577875867,Фото: Алексей Сухоруков / РИА Новости В Сарапу...,Один из пострадавших скончался,Фото: Алексей Сухоруков / РИА Новости В ... пр...
2,943881,https://lenta.ru/news/2020/01/01/msc/,В новогоднюю ночь в Москве похитили семилетнюю...,1577892180,1577895279,1,1,0,0,37,[134],https://icdn.lenta.ru/images/2020/01/01/18/202...,1577892180,Подозреваемая в похищении ребенка в Москве Фот...,Ребенка увела неизвестная женщина,Подозреваемая в похищении ребенка в ... 31 дек...
3,943883,https://lenta.ru/news/2020/01/01/zastrelil/,Полицейский застрелил избившего жену россиянина,1577894462,1577894462,1,1,0,0,37,[136],https://icdn.lenta.ru/images/2020/01/01/18/202...,1577894462,Фото: Максим Блинов / РИА Новости В Камышлове ...,"По данным МВД, табельное оружие было применено...",Фото: Максим Блинов / РИА Новости В ... семейн...
4,943884,https://lenta.ru/news/2020/01/01/podrobnosti/,Появились подробности похищения семилетней дев...,1577895589,1577895589,1,1,0,0,37,[],https://icdn.lenta.ru/images/2020/01/01/19/202...,1577895589,Подозреваемая в похищении ребенка в Москве Фот...,Родители ребенка не заперли дверь в квартиру,Подозреваемая в похищении ребенка в ... девочк...


In [ ]:
df4 = tbl[['text', 'bloc']]
df4.head

<bound method NDFrame.head of                                                     text  bloc
0      Фото: Наталья Селиверстова / РИА Новости Состо...    37
1      Фото: Алексей Сухоруков / РИА Новости В Сарапу...    37
2      Подозреваемая в похищении ребенка в Москве Фот...    37
3      Фото: Максим Блинов / РИА Новости В Камышлове ...    37
4      Подозреваемая в похищении ребенка в Москве Фот...    37
...                                                  ...   ...
29642  Фото: Иван Водопьянов / Коммерсантъ Алевтина З...    37
29643  Фото: Andrey_Vasiliskov / Shutterstock / Fotod...    37
29644  Здание ФСБ на Лубянской площади в Москве Фото:...    37
29645  Фото: Maksim Konstantinov / Global Look Press ...    37
29646  Фото: Konstantin Kokoshkin/Russian Look Марина...    37

[29647 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "5"
dateFrom = "2010-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '5', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
26079


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943836,https://lenta.ru/news/2020/01/01/sleep/,Объяснена опасность неправильного сна,1577839980,1577964161,1,1,0,0,5,[16],https://icdn.lenta.ru/images/2020/01/01/03/202...,1577839980,Фото: Vladislav Muslakov / Unsplash Спящие бол...,Спящие слишком мало или слишком много более по...,"Фото: Vladislav Muslakov / Unsplash ..., чем т..."
1,943844,https://lenta.ru/news/2020/01/01/pohmel/,Названы облегчающие похмелье средства,1577857380,1577963225,1,1,0,0,5,[16],https://icdn.lenta.ru/images/2020/01/01/08/202...,1577857380,Фото: Михаил Фомичев / РИА Новости Исследовате...,"Среди них спаржа, газированный сок лайма и лим...","Фото: Михаил Фомичев / РИА Новости ..., сообща..."
2,943911,https://lenta.ru/news/2020/01/02/baltika/,На Балтику перебросят «охотников на акул»,1577922480,1577978732,1,1,0,0,5,[13],https://icdn.lenta.ru/images/2020/01/02/02/202...,1577922480,Фото: Виталий Аньков / РИА Новости Первые моде...,Вертолеты Ка-27М поступят на вооружение Балтий...,Фото: Виталий Аньков / РИА Новости ... Балтийс...
3,943916,https://lenta.ru/news/2020/01/02/rassol/,Опровергнута польза рассола при похмелье,1577928540,1577936898,1,1,0,0,5,[195],https://icdn.lenta.ru/images/2020/01/02/04/202...,1577928540,Фото: Виталий Аньков / РИА Новости Маринованны...,Маринованные овощи не могут оказать влияния на...,Фото: Виталий Аньков / РИА Новости ... последс...
4,943923,https://lenta.ru/news/2020/01/02/sposob/,Врач предупредил о смертельно опасном способе ...,1577937124,1577937124,1,1,0,0,5,[195],https://icdn.lenta.ru/images/2020/01/02/06/202...,1577937124,Фото: Евгений Биятов / РИА Новости Главный вне...,"Евгений Брюн заявил, что опохмеляться нельзя",Фото: Евгений Биятов / РИА Новости ... алкогол...


In [ ]:
df5 = tbl[['text', 'bloc']]
df5.head

<bound method NDFrame.head of                                                     text  bloc
0      Фото: Vladislav Muslakov / Unsplash Спящие бол...     5
1      Фото: Михаил Фомичев / РИА Новости Исследовате...     5
2      Фото: Виталий Аньков / РИА Новости Первые моде...     5
3      Фото: Виталий Аньков / РИА Новости Маринованны...     5
4      Фото: Евгений Биятов / РИА Новости Главный вне...     5
...                                                  ...   ...
26074  Фото: Unsplash Олег Давыдов Корпорация Microso...     5
26075  Фото: Kacper Pempel / Reuters Марина Совина Со...     5
26076  Фото: Unsplash Виктория Кондратьева Компания C...     5
26077  Фото: Unsplash Константин Лысяков Французские ...     5
26078  Архивное фото Фото: Администрация Президента У...     5

[26079 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "8"
dateFrom = "2010-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '8', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

,rightcol,docid,lastmodtime,part,title,type,url,tags,bloc,domain,modified,image_url,text,status,pubdate,snippet
0,"Среди мужчин звание лучшего атлета планеты, та...",187,1670590774,0,Елена Исинбаева во второй раз подряд признана ...,1,https://lenta.ru/news/2010/01/01/thebestof/,[],8,1,1262332426,https://icdn.lenta.ru/images/0000/0282/0000028...,Елена Исинбаева. Фото (c)AFP Российская прыгун...,0,1262332426,Елена Исинбаева. Фото (c)AFP Российская прыгун...
1,Главный тренер сборной США по хоккею назвал со...,272,1670590779,0,Объявлен состав сборной США по хоккею на Олимп...,1,https://lenta.ru/news/2010/01/02/hockey/,[],8,1,1262396695,https://icdn.lenta.ru/images/0000/0282/0000028...,Игроки сборной США по хоккею. Фото из архива (...,0,1262396695,Игроки сборной США по хоккею. Фото из архива (...
2,Московские футбольные клубы готовы предложить ...,361,1670590786,0,"ЦСКА и ""Динамо"" вступили в борьбу за Андрея Во...",1,https://lenta.ru/news/2010/01/02/voronin/,[],8,1,1262465193,https://icdn.lenta.ru/images/0000/0282/0000028...,Андрей Воронин (слева). Фото (c)AFP Нападающим...,0,1262465193,Андрей Воронин (слева). Фото (c)AFP Нападающим...
3,Врачи мюнхенской клиники провели российской бо...,366,1670590787,0,Бобслеистке Скворцовой сделана вторая операция...,1,https://lenta.ru/news/2010/01/02/skvortsova/,[],8,1,1262469789,https://icdn.lenta.ru/images/0000/0282/0000028...,"Ирина Скворцова. Кадр канала ""Спорт"", архив 2 ...",0,1262469789,"Ирина Скворцова. Кадр канала ""Спорт"", архив 2 ..."
4,"Француз разбился на квадроцикле, катаясь с доч...",464,1670590794,0,Попавшему в аварию биатлонисту Рафаэлю Пуаре п...,1,https://lenta.ru/news/2010/01/03/poiree/,[],8,1,1262531418,https://icdn.lenta.ru/images/0000/0282/0000028...,Рафаэль Пуаре. Фото (c)AFP Знаменитому биатлон...,0,1262531418,Рафаэль Пуаре. Фото (c)AFP Знаменитому биатлон...


In [ ]:
df6 = tbl[['text', 'bloc']]
df6.head

<bound method NDFrame.head of                                                     text  bloc
0      Али Исаев (сверху) Фото: Gregory Payan / AP Ро...     8
1      Кадр: Twitter PFL MMA Российский боец смешанно...     8
2      Евгений Малкин Фото: Christopher Hanewinckel /...     8
3      Фото: Алексей Витвицкий / РИА Новости Всемирно...     8
4      Александр Большунов Фото: Алексей Филиппов / Р...     8
...                                                  ...   ...
29396  Архивное фото Фото: Александр Вильф / РИА Ново...     8
29397  Фото: Geoff Burke / USA TODAY Sports / Reuters...     8
29398  Фото: Marcos Michael Livepic / Reuters Алексей...     8
29399  Фото: Владимир Астапкович / РИА Новости Алексе...     8
29400  Игорь Акинфеев Фото: Alexander Kulebyakin / Gl...     8

[29401 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "48"
dateFrom = "2020-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '48', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
19909


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943943,https://lenta.ru/news/2020/01/02/fatalcrashes/,Названо число жертв авиакатастроф в 2019 году,1577955420,1577956165,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2020/01/02/11/202...,1577955420,Место крушения самолета Fokker 100 казахстанск...,Из-за несчастных случаев погибли 257 человек,Место крушения самолета Fokker 100 ... в 2019 ...
1,943965,https://lenta.ru/news/2020/01/02/crashairport/,Последствия столкновения двух самолетов попали...,1577966520,1577968150,1,1,0,7,48,[4],https://icdn.lenta.ru/images/2020/01/02/14/202...,1577966520,Фото: Twitter / @avioesemusicas Последствия ст...,Boeing 777 задел хвостовую часть Airbus A321,Фото: Twitter / @avioesemusicas ... в аэропорт...
2,944014,https://lenta.ru/news/2020/01/03/best_avia/,Названа самая безопасная авиакомпания в мире,1578011400,1578030890,1,1,0,0,48,[116],https://icdn.lenta.ru/images/2020/01/03/03/202...,1578011400,Фото: Flickr / jason goulding Австралийская ав...,Qantas стала наиболее надежным перевозчиком,Фото: Flickr / jason goulding ... экспертов по...
3,944019,https://lenta.ru/news/2020/01/03/aeroflot/,«Аэрофлот» стал самой пунктуальной авиакомпани...,1578028860,1578029392,1,1,0,0,48,[159],https://icdn.lenta.ru/images/2020/01/03/08/202...,1578028860,Фото: Алексей Филиппов / РИА Новости Крупнейши...,"Российский авиаперевозчик выполнил 86,7 процен...",Фото: Алексей Филиппов / РИА Новости ... пункт...
4,944050,https://lenta.ru/news/2020/01/03/neudacha/,Пассажир бросил монетки в двигатель самолета н...,1578046920,1578047548,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2020/01/03/13/202...,1578046920,Фото: Reuters / Regis Duvignau Пассажир из Кит...,Лу Чао был оштрафован на 17 тысяч долларов,Фото: Reuters / Regis Duvignau Пассажир ... су...


In [ ]:
df7 = tbl[['text', 'bloc']]
df7.head

<bound method NDFrame.head of                                                     text  bloc
0      Место крушения самолета Fokker 100 казахстанск...    48
1      Фото: Twitter / @avioesemusicas Последствия ст...    48
2      Фото: Flickr / jason goulding Австралийская ав...    48
3      Фото: Алексей Филиппов / РИА Новости Крупнейши...    48
4      Фото: Reuters / Regis Duvignau Пассажир из Кит...    48
...                                                  ...   ...
19904  Фото: Alexander Welscher / Globallookpress.com...    48
19905  Фото: Нина Падалко / Пресс-служба ОАК / РИА Но...    48
19906  Кадр: CBS News Зарина Дзагоева Пассажирский са...    48
19907  Фото: Anna Kudriavtseva / Reuters Зарина Дзаго...    48
19908  Фото: Unsplash Зарина Дзагоева Пьяный пассажир...    48

[19909 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "87"
dateFrom = "2020-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '87', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
8306


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1205238,https://lenta.ru/news/2022/02/23/heart/,Названы продукты для здоровья сердца,1645567200,1648746997,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/23/00/202...,1645567200,Фото: Pixabay.com Марина Совина Употребление н...,Названы продукты для здоровья сердца,"Фото: Pixabay.com Марина Совина ... диетологи,..."
1,1205325,https://lenta.ru/news/2022/02/23/ice_cream/,Россиянам развеяли популярный миф о лечении бо...,1645590960,1648747025,1,1,0,0,87,[355],https://icdn.lenta.ru/images/2022/02/23/06/202...,1645590960,Фото: Pixabay Марина Совина Врач-отоларинголог...,Россиянам развеяли популярный миф о лечении бо...,Фото: Pixabay Марина Совина Врач-... лекарство...
2,1205331,https://lenta.ru/news/2022/02/23/diabet/,Названы способные предотвратить диабет продукты,1645592820,1648747044,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/23/08/202...,1645592820,"Фото: Pixabay Марина Совина Семейный врач, спе...",Названы способные предотвратить диабет продукты,Фото: Pixabay Марина Совина Семейный ... вспят...
3,1205841,https://lenta.ru/news/2022/02/25/broccoli/,Названо средство для крепкого сна даже после ч...,1645769520,1648747186,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/24/12/202...,1645769520,Фото: Louis Hansel / Unsplash Олег Парамонов А...,Названо средство для крепкого сна даже после ч...,Фото: Louis Hansel / Unsplash Олег ... крепко ...
4,1205881,https://lenta.ru/news/2022/02/25/4veg/,Диетолог рассказала о вредном воздействии на м...,1645776720,1648747198,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/24/12/202...,1645776720,Фото: Shutterstock Никита Савин Диетолог из СШ...,Диетолог рассказала о вредном воздействии на м...,Фото: Shutterstock Никита Савин Диетолог ... о...


In [ ]:
df8 = tbl[['text', 'bloc']]
df8.head

<bound method NDFrame.head of                                                    text  bloc
0     Фото: Pixabay.com Марина Совина Употребление н...    87
1     Фото: Pixabay Марина Совина Врач-отоларинголог...    87
2     Фото: Pixabay Марина Совина Семейный врач, спе...    87
3     Фото: Louis Hansel / Unsplash Олег Парамонов А...    87
4     Фото: Shutterstock Никита Савин Диетолог из СШ...    87
...                                                 ...   ...
8301  Фото: Kateryna Hliznitsova / Unsplash Екатерин...    87
8302  Фото: Анна Майорова / «Лента.ру» Кристина Есик...    87
8303  Фото: Farhad Ibrahimzade / Unsplash Анатолий А...    87
8304  Фото: Jay Wennington / Unsplash Екатерина Смир...    87
8305  Фото: Ruslan Lvoff / Shutterstock / Fotodom Ве...    87

[8306 rows x 2 columns]>

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = '1'
bloc = "12"
dateFrom = "2010-01-01"
dateTo = "2024-02-01"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2010-01-01', 'dateTo': '2024-02-01', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '16', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Parsing articles from 2024-01-18 to 2024-02-01
Finish
0


<ipython-input-5-2bfa07dc550d>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


""


In [ ]:
df9 = tbl[['text', 'bloc']]
df9.head

In [ ]:
new_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8])
new_df.head

<bound method NDFrame.head of                                                    text  bloc
0     Фото:  Юрий Мартьянов /« Коммерсантъ» В России...     1
1     Фото: Александр Коряков / «Коммерсантъ» Автомо...     1
2     Фото: ИА PrimaMedia Фигуру мыши, которая стоял...     1
3     Фото: Константин Кокошкин / Globallookpress.co...     1
4     Фото: Сергей Киселев / АГН «Москва» В России в...     1
...                                                 ...   ...
8301  Фото: Kateryna Hliznitsova / Unsplash Екатерин...    87
8302  Фото: Анна Майорова / «Лента.ру» Кристина Есик...    87
8303  Фото: Farhad Ibrahimzade / Unsplash Анатолий А...    87
8304  Фото: Jay Wennington / Unsplash Екатерина Смир...    87
8305  Фото: Ruslan Lvoff / Shutterstock / Fotodom Ве...    87

[225441 rows x 2 columns]>

In [ ]:
new_df = new_df.rename(columns={ 'bloc': 'topic'})
new_df.head

<bound method NDFrame.head of                                                    text  topic
0     Фото:  Юрий Мартьянов /« Коммерсантъ» В России...      1
1     Фото: Александр Коряков / «Коммерсантъ» Автомо...      1
2     Фото: ИА PrimaMedia Фигуру мыши, которая стоял...      1
3     Фото: Константин Кокошкин / Globallookpress.co...      1
4     Фото: Сергей Киселев / АГН «Москва» В России в...      1
...                                                 ...    ...
8301  Фото: Kateryna Hliznitsova / Unsplash Екатерин...     87
8302  Фото: Анна Майорова / «Лента.ру» Кристина Есик...     87
8303  Фото: Farhad Ibrahimzade / Unsplash Анатолий А...     87
8304  Фото: Jay Wennington / Unsplash Екатерина Смир...     87
8305  Фото: Ruslan Lvoff / Shutterstock / Fotodom Ве...     87

[225441 rows x 2 columns]>

In [ ]:
new_df = new_df.replace({'topic' : {1: 0, 4: 1, 37: 2, 8: 4, 5: 8, 48 : 7, 87 : 5, 12 : 6 }})
new_df.head

<bound method NDFrame.head of                                                    text  topic
0     Фото:  Юрий Мартьянов /« Коммерсантъ» В России...      0
1     Фото: Александр Коряков / «Коммерсантъ» Автомо...      0
2     Фото: ИА PrimaMedia Фигуру мыши, которая стоял...      0
3     Фото: Константин Кокошкин / Globallookpress.co...      0
4     Фото: Сергей Киселев / АГН «Москва» В России в...      0
...                                                 ...    ...
8301  Фото: Kateryna Hliznitsova / Unsplash Екатерин...      5
8302  Фото: Анна Майорова / «Лента.ру» Кристина Есик...      5
8303  Фото: Farhad Ibrahimzade / Unsplash Анатолий А...      5
8304  Фото: Jay Wennington / Unsplash Екатерина Смир...      5
8305  Фото: Ruslan Lvoff / Shutterstock / Fotodom Ве...      5

[225441 rows x 2 columns]>

In [ ]:
new_df.nunique()

text     225349
topic         8
dtype: int64

In [ ]:
tbl_new = new_df[~new_df.text.isna()]

print(len(new_df), len(tbl_new))

225441 225441


In [ ]:
X = tbl_new[['text']]
y = tbl_new['topic']

X.shape

(225441, 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((169080, 1), (56361, 1))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec=CountVectorizer(max_df=100000,max_features=1000000)
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=20000, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(169080, 427444)
              precision    recall  f1-score   support

           0       0.87      0.90      0.88      9685
           1       0.95      0.95      0.95      9234
           2       0.96      0.95      0.96      7378
           3       0.94      0.94      0.94      9017
           4       1.00      0.99      0.99      7432
           5       0.97      0.96      0.97      2096
           7       0.98      0.96      0.97      5007
           8       0.97      0.95      0.96      6512

    accuracy                           0.95     56361
   macro avg       0.95      0.95      0.95     56361
weighted avg       0.95      0.95      0.95     56361



In [ ]:
Test = pd.read_csv("/content/test_news.csv")
Test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer(max_df=100000,max_features=1000000)
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=20000, random_state=42)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [ ]:
print(classification_report(pred))

TypeError: ignored

In [ ]:
pred[:10], len(pred)

(array([0, 1, 4, 0, 4, 3, 2, 3, 3, 3]), 26275)

In [ ]:
subm = pd.read_csv("base_submission_news.csv")
subm.head()

,topic,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [ ]:
subm['topic'] = pred

subm.to_csv("my_baseline_submission.csv", index=False)

In [ ]:
subm['topic'].value_counts(normalize=True)

0    0.344738
1    0.176327
3    0.140019
4    0.098573
2    0.095794
8    0.071018
7    0.057469
5    0.016061
Name: topic, dtype: float64

In [ ]:
!kaggle competitions submit -c news-scraping-competition -f my_baseline_submission.csv -m "Message"

100% 194k/194k [00:01<00:00, 115kB/s]  
Successfully submitted to Предсказание новостных тематик

In [ ]:
class rbc_parser:
    def __init__(self):
        pass


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/v10/search/ajax/?\
        project={0}&\
        category={1}&\
        dateFrom={2}&\
        dateTo={3}&\
        offset={4}&\
        limit={5}&\
        query={6}&\
        material={7}'.format(param_dict['project'],
                            param_dict['category'],
                            param_dict['dateFrom'],
                            param_dict['dateTo'],
                            param_dict['offset'],
                            param_dict['limit'],
                            param_dict['query'],
                            param_dict['material'])

        return url


    def _get_search_table(self, param_dict: dict,
                          includeText: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей

        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if includeText and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()

        return search_table.sort_values('publish_date_t', ignore_index=True)


    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None

        return overview, text

    def get_articles(self,
                     param_dict,
                     time_step = 7,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [ ]:
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2020-01-01'
dateTo = "2024-01-04"
offset = 0
limit = 100

if use_parser == "РБК":
    param_dict = {'query'   : query,
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.01.2020', 'dateTo': '04.01.2024', 'offset': '0', 'limit': '100', 'material': ''}


In [ ]:
assert use_parser == "РБК"
parser = rbc_parser()
table = parser.get_articles(param_dict=param_dict,
                            time_step = 7, # Шаг - 7 дней, можно больше,
                                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                            save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                            save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Parsing articles from 01.01.2020 to 08.01.2020


JSONDecodeError: ignored

In [ ]:
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               includeText = True) # Парсить текст статей
print(len(tbl))
tbl.head()

JSONDecodeError: ignored

In [ ]:
%%capture
!pip install selenium

In [ ]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
SLEEP = 2
DEPTH = 200
BASE_URL = "https://realty.ria.ru/"
TOPICS = ["realty"]

In [ ]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None

In [ ]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

            # push to list 20 next articles
            driver.execute_script(
                "document.getElementsByClassName('list-more')[0].click()"
            )
            time.sleep(1)

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 1200)"
                    )
                    time.sleep(1)
                except:
                    pass

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

In [ ]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "list-item__image"})["href"]

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article.id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article.id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    # article number of views
    # article.views = int(obj.find('span', {'class': 'statistic__item m-views'}).text)

    return article

In [ ]:
# get pages and topics
pages, topics_order = get_pages()

100%|██████████| 1/1 [03:38<00:00, 218.31s/it]


In [ ]:
with open("ria_pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write("%s\n" % page)

In [ ]:
len(pages)

0

In [ ]:
data, topics_order_fixed = [], []

for num, page in enumerate(tqdm(pages)):
    try:
        res = parse_page(page) # ваш код здесь
        data.append(res) # ваш код здесь

        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

NameError: ignored

In [ ]:
data = pd.read_csv("/content/test_news.csv")
data.head

<bound method NDFrame.head of                                                  content
0      Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1      В начале февраля 2023 года в Пушкинском районе...
2      Фото: Andy Bao / Getty Images Анастасия Борисо...
3      Если вы хотели, но так и не съездили на море л...
4      Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...                                                  ...
26270  Фото: РИА Новости Алевтина Запольская Главное ...
26271  Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272  Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273  Владимир Зеленский Фото: Yves Herman / Reuters...
26274  Фото: President of the Republic of Azerbaijan ...

[26275 rows x 1 columns]>

In [ ]:
data2 = pd.read_csv("/content/base_submission_news.csv")
data2.head

<bound method NDFrame.head of        topic  index
0          0      0
1          0      1
2          0      2
3          0      3
4          0      4
...      ...    ...
26270      0  26270
26271      0  26271
26272      0  26272
26273      0  26273
26274      0  26274

[26275 rows x 2 columns]>

In [ ]:
data.iloc[0]['content']

'Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обороны ДНР Игорю Стрелкову (Гиркину) пришлось уйти в подполье, чтобы встретиться в Петербурге с тридцатью поклонниками. В создании нужной атмосферы были задействованы: один автозак Росгвардии, один кинолог, одна собака, четыре полицейские машины и чертова дюжина силовиков.\n\n\nЗа последние семь лет Стрелков, он же Гиркин, выступал в Петербурге столь часто, что это само по себе перестало быть событием. Предыдущий раз он собирал аншлаг в той же «Листве» в январе 2023-го.\n\n\nКнижная лавка «Листва» на Литейном проспекте принадлежит издательству «Черная сотня», в самом названии которого содержится указание на правый характер сподвижников. Себя считают обществом патриотов. На регулярной основе здесь проходят «встречи с читателями». Военкор Владлен Татарский приходил сюда за несколько часов до взорванной на Университетской набережной бомбы. Бывала в «Листве» и Дарья Дугина. Весной здесь выступал самый известный (между путчем 1991-го и маршем Приг